In [70]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [54]:
dataset = pd.read_excel('World_Bank_Projects_downloaded_6_13_2022.xls')

In [55]:
#types of all columns in the dataframe
dataset.dtypes

id                           object
regionname                   object
countryname                  object
projectstatusdisplay         object
project_name                 object
pdo                          object
impagency                    object
cons_serv_reqd_ind           object
url                          object
boardapprovaldate            object
closingdate                  object
projectfinancialtype         object
curr_project_cost           float64
curr_ibrd_commitment          int64
curr_ida_commitment           int64
curr_total_commitment         int64
grantamt                      int64
borrower                     object
lendinginstr                 object
envassesmentcategorycode     object
esrc_ovrl_risk_rate          object
sector1                      object
sector2                      object
sector3                      object
theme1                       object
theme2                       object
dtype: object

In [56]:
#There are 21487 rows in the dataset

len(dataset)

21487

In [57]:
#trying to find out what values are there in the dataframe, types, formatting
dataset.head()

,id,regionname,countryname,projectstatusdisplay,project_name,pdo,impagency,cons_serv_reqd_ind,url,boardapprovaldate,...,grantamt,borrower,lendinginstr,envassesmentcategorycode,esrc_ovrl_risk_rate,sector1,sector2,sector3,theme1,theme2
0,P169983,Africa East,Republic of Angola,Active,Third Angola Growth and Inclusion Development ...,The development objective is to support the Go...,Ministry of Finance of the Republic of Angola,NaN,http://projects.worldbank.org/P169983/null?lan...,2022-03-25T00:00:00Z,...,0,REPUBLIC OF ANGOLA,Development Policy Lending,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P173711,Africa East,Republic of Madagascar,Active,Connecting Madagascar for Inclusive Growth,The Project Development Objective is to improv...,Road Agency,NaN,http://projects.worldbank.org/P173711/null?lan...,2022-03-25T00:00:00Z,...,0,REPUBLIC OF MADAGASCAR,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P175747,Middle East and North Africa,Kingdom of Morocco,Active,Resilient and Sustainable Water in Agriculture,The project development objectives (PDO) are t...,Ministry of Agriculture - Directorate of Irrig...,NaN,http://projects.worldbank.org/P175747/null?lan...,2022-03-25T00:00:00Z,...,0,Kingdom of Morocco,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P175792,Africa East,Republic of Malawi,Active,Invigorating local market & competition in Pub...,Invigorating local market & competition in Pub...,Malawi - Public Procurement and Disposal of As...,NaN,http://projects.worldbank.org/P175792/null?lan...,2022-03-25T00:00:00Z,...,410000,Republic of Malawi,Investment Project Financing,C,NaN,NaN,NaN,NaN,NaN,NaN
4,P177073,East Asia and Pacific,Federated States of Micronesia,Active,Federated States of Micronesia Strategic Clima...,To improve the climate resilience of the Recip...,"Department of Transportation, Communications &...",NaN,http://projects.worldbank.org/P177073/null?lan...,2022-03-25T00:00:00Z,...,0,Federated States of Micronesia,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
dataset.describe()

,curr_project_cost,curr_ibrd_commitment,curr_ida_commitment,curr_total_commitment,grantamt
count,2.134000e+04,2.148700e+04,2.148700e+04,2.148700e+04,2.148700e+04
mean,1.573027e+08,4.537050e+07,2.719748e+07,7.256798e+07,1.924198e+06
std,1.084423e+09,1.227862e+08,6.946621e+07,1.336311e+08,3.925397e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+07,0.000000e+00,0.000000e+00,5.000000e+06,0.000000e+00
50%,3.260000e+07,0.000000e+00,0.000000e+00,2.500000e+07,0.000000e+00
75%,1.013250e+08,3.150000e+07,2.200000e+07,8.400000e+07,0.000000e+00
max,9.300700e+10,3.750000e+09,1.375000e+09,3.750000e+09,5.330000e+09


In [59]:
#so we have Active, Closed, Pipeline and Dropped types, along with NaN
dataset.projectstatusdisplay.unique()


array(['Active', 'Closed', nan, 'Pipeline', 'Dropped'], dtype=object)

In [60]:
dataset.regionname.unique()

array(['Africa East', 'Middle East and North Africa',
       'East Asia and Pacific', 'Africa West',
       'Latin America and Caribbean', 'South Asia',
       'Europe and Central Asia', 'Other', 'Africa', nan], dtype=object)

In [61]:
#finding out NaN values for project status display: there are only three, so we may exclude them.
dataset[dataset['projectstatusdisplay'].isna()]

,id,regionname,countryname,projectstatusdisplay,project_name,pdo,impagency,cons_serv_reqd_ind,url,boardapprovaldate,...,grantamt,borrower,lendinginstr,envassesmentcategorycode,esrc_ovrl_risk_rate,sector1,sector2,sector3,theme1,theme2
3301,P156335,East Asia and Pacific,Pacific Islands,NaN,PACIFIC RESILIENCE PROGRAM,The objective of the Program is to strengthen ...,NaN,NaN,http://projects.worldbank.org/P156335/null?lan...,2015-06-19T00:00:00Z,...,900000,NaN,Investment Project Financing,B,NaN,"Other Water Supply, Sanitation and Waste Manag...",Public Administration - Financial Sector,Other Transportation,Natural disaster management,Climate change
4752,P121213,Africa West,Republic of Liberia,NaN,DROPPED - LR: Road Asset Management Project - ...,The objective is to improve transport efficien...,NaN,NaN,http://projects.worldbank.org/P121213/liberia-...,2012-08-07T00:00:00Z,...,113000000,NaN,Specific Investment Loan,B,NaN,Rural and Inter-Urban Roads,NaN,NaN,Infrastructure services for private sector dev...,NaN
18367,P114115,Africa West,Republic of Guinea-Bissau,NaN,Education for All - Fast Track Initiative Program,The Project Development Objective is to improv...,NaN,NaN,http://projects.worldbank.org/P114115/educatio...,NaN,...,12000000,NaN,Specific Investment Loan,B,NaN,Primary Education,Secondary Education,NaN,Education for all,NaN


In [62]:
dataset[dataset['curr_project_cost'].isna()]['curr_project_cost'] = -dataset['curr_project_cost'].max()

/tmp/ipykernel_33990/155088899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[dataset['curr_project_cost'].isna()]['curr_project_cost'] = -dataset['curr_project_cost'].max()


In [63]:
#removing rows from the dataset where projectstatusdisplay is nan
dataset = dataset[dataset['projectstatusdisplay'].notna()]

In [64]:
#now, encoding pipeline, closed and active as 1, dropped as 0
#Assuming that closed project means that the projects have been completed...

dataset['projectstatus'] = dataset.apply(lambda row: 0 if row.projectstatusdisplay == 0 else 1, axis = 1)

/tmp/ipykernel_33990/3133648340.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['projectstatus'] = dataset.apply(lambda row: 0 if row.projectstatusdisplay == 0 else 1, axis = 1)


In [65]:
#encoding the categorical variables
label_encoder = preprocessing.LabelEncoder()
dataset['regioncode'] = label_encoder.fit_transform(dataset.regionname)
dataset['countrycode'] = label_encoder.fit_transform(dataset['countryname'])
dataset['countrycode'] = label_encoder.fit_transform(dataset['countryname'])
dataset['financecode'] = label_encoder.fit_transform(dataset['projectfinancialtype'])
dataset['borrowercode'] = label_encoder.fit_transform(dataset['borrower'])
dataset['lendinginstrcode'] = label_encoder.fit_transform(dataset['lendinginstr'])
dataset['envassesmentcategorycode'] = label_encoder.fit_transform(dataset['envassesmentcategorycode'])
dataset['esrc_ovrl_risk_rate'] = label_encoder.fit_transform(dataset['esrc_ovrl_risk_rate'])
dataset['sector1'] = label_encoder.fit_transform(dataset['sector1'])
dataset['sector2'] = label_encoder.fit_transform(dataset['sector2'])
dataset['sector3'] = label_encoder.fit_transform(dataset['sector3'])
dataset['theme1'] = label_encoder.fit_transform(dataset['theme1'])
dataset['theme2'] = label_encoder.fit_transform(dataset['theme2'])

In [66]:
# print(dataset.head())
print(dataset.envassesmentcategorycode.unique(),
dataset.esrc_ovrl_risk_rate.unique(),
dataset.sector1.unique(),
dataset.sector2.unique(),
dataset.sector3.unique(),
dataset.theme1.unique(),
dataset.theme2.unique()
,sep = '\n\n')

[8 2 1 0 6 4 7 3 5]

[4 3 1 0 2]

[159 145 146  76  94  83 148 111  93 116 149 144 139 130  82 125 158 118
 138  91 157 112  99 127  98 126  87  85  80  86  81 133 135 101  89 154
  77 128 117 134 110 140 102 141 100 113 151 115 156 109 131 103 104 152
 132  95 129 136 122 107 114 153 108 137  75 124 106 143 142  78 105 147
  33  26 123  36  16  41  54  29  88  35  48 120 119  96 155 150 121  79
  12  97  38  64  63  56  18  55  43  92  73  71  23   1   3  24   8  84
  67  27  21  30  60  13  14  70  19  66  31  34  57  40  68  62  53  22
  17  11   5  47  90  72  58  39   2  25   7   6  45  44  69  50  32  59
  74  37   4  10  20  28  51  65  61  42   9  52  46   0  15  49]

[102  99  29  33  72  42  58  69  92  40  36  76  91  68  52  26  31  75
  87  82 101  20  70  74  19  35  60  41  57  88 100  54  67  71  86  80
  44  30  49  97  59  45  73  53  24  43  95  96  55  77  56  18  37  51
  28  25  23  81  79  64  94  89  46  50  83  78  84  48  90  66  32  85
  47  21  65   2  38  6

In [67]:
#transforming nan values to 0 in envassesmentcategorycode, esrc_ovrl_risk_rate, sector1, sector2, sector3, theme1, theme2

In [68]:
dataset.isna().any()

id                          False
regionname                   True
countryname                  True
projectstatusdisplay        False
project_name                False
pdo                          True
impagency                    True
cons_serv_reqd_ind           True
url                         False
boardapprovaldate            True
closingdate                  True
projectfinancialtype         True
curr_project_cost            True
curr_ibrd_commitment        False
curr_ida_commitment         False
curr_total_commitment       False
grantamt                    False
borrower                     True
lendinginstr                 True
envassesmentcategorycode    False
esrc_ovrl_risk_rate         False
sector1                     False
sector2                     False
sector3                     False
theme1                      False
theme2                      False
projectstatus               False
regioncode                  False
countrycode                 False
financecode   

In [81]:
dataset.dtypes

id                           object
regionname                   object
countryname                  object
projectstatusdisplay         object
project_name                 object
pdo                          object
impagency                    object
cons_serv_reqd_ind           object
url                          object
boardapprovaldate            object
closingdate                  object
projectfinancialtype         object
curr_project_cost           float64
curr_ibrd_commitment          int64
curr_ida_commitment           int64
curr_total_commitment         int64
grantamt                      int64
borrower                     object
lendinginstr                 object
envassesmentcategorycode      int64
esrc_ovrl_risk_rate           int64
sector1                       int64
sector2                       int64
sector3                       int64
theme1                        int64
theme2                        int64
projectstatus                 int64
regioncode                  

In [87]:
#taking transformed and relevant columns, rejecting others
req_cols = ['project_name', 'pdo', 'impagency', 'curr_project_cost', 'curr_ibrd_commitment', 'curr_ida_commitment', 'curr_total_commitment', 'grantamt', 'envassesmentcategorycode', 'esrc_ovrl_risk_rate', 'sector1', 'sector2', 'sector3', 'theme1', 'theme2', 'regioncode', 'countrycode', 'financecode', 'borrowercode', 'lendinginstrcode', 'projectstatus']
dataset = pd.DataFrame(dataset, columns = req_cols)

In [94]:
dataset.isna().any()

project_name                False
pdo                          True
impagency                    True
curr_project_cost            True
curr_ibrd_commitment        False
curr_ida_commitment         False
curr_total_commitment       False
grantamt                    False
envassesmentcategorycode    False
esrc_ovrl_risk_rate         False
sector1                     False
sector2                     False
sector3                     False
theme1                      False
theme2                      False
regioncode                  False
countrycode                 False
financecode                 False
borrowercode                False
lendinginstrcode            False
projectstatus               False
dtype: bool

In [96]:
#since there still exist na values, fill them with empty strings (textual data)
dataset = dataset.fillna('')
dataset.isna().any()

project_name                False
pdo                         False
impagency                   False
curr_project_cost           False
curr_ibrd_commitment        False
curr_ida_commitment         False
curr_total_commitment       False
grantamt                    False
envassesmentcategorycode    False
esrc_ovrl_risk_rate         False
sector1                     False
sector2                     False
sector3                     False
theme1                      False
theme2                      False
regioncode                  False
countrycode                 False
financecode                 False
borrowercode                False
lendinginstrcode            False
projectstatus               False
dtype: bool

In [91]:
# train test split
train, test = train_test_split(dataset, test_size = 0.2)

In [97]:
print(len(train), len(test), sep = '\n')

17187
4297
